In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from citipy import citipy
# Import the API key.
from config import g_key

from datetime import datetime
import time
from scipy.stats import linregress

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ambon,FR,2020-01-04 00:15:33,47.55,-2.56,46.40,93,4,4.70
1,1,Chiredzi,ZW,2020-01-04 00:14:11,-21.05,31.67,73.45,75,0,5.41
2,2,Hilo,US,2020-01-04 00:15:34,19.71,-155.08,82.40,58,20,8.05
3,3,Hastings,NZ,2020-01-04 00:15:34,-39.64,176.84,59.54,71,66,11.30
4,4,Kawalu,ID,2020-01-04 00:15:34,-7.38,108.21,73.17,85,100,2.59


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Hilo,US,2020-01-04 00:15:34,19.71,-155.08,82.40,58,20,8.05
17,17,Los Tangos,HN,2020-01-04 00:15:37,15.15,-88.68,80.60,78,40,0.72
20,20,Hobart,AU,2020-01-04 00:15:38,-42.88,147.33,82.00,24,92,6.93
23,23,Rikitea,PF,2020-01-04 00:14:04,-23.12,-134.97,79.72,76,100,10.67
26,26,Bambous Virieux,MU,2020-01-04 00:15:40,-20.34,57.76,77.00,94,40,11.14
32,32,Carutapera,BR,2020-01-04 00:15:42,-1.20,-46.02,81.75,80,90,10.47
34,34,San Patricio,PY,2020-01-04 00:15:42,-26.98,-56.83,79.86,49,10,6.96
35,35,Cockburn Town,TC,2020-01-04 00:15:42,21.46,-71.14,79.30,80,23,17.83
42,42,Majene,ID,2020-01-04 00:11:49,-3.54,118.97,81.00,67,97,5.99
44,44,Atuona,PF,2020-01-04 00:15:44,-9.80,-139.03,80.64,71,4,14.97


In [12]:
preferred_cities_df.count()

City_ID       136
City          136
Country       136
Date          136
Lat           136
Lng           136
Max Temp      136
Humidity      136
Cloudiness    136
Wind Speed    136
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,82.40,19.71,-155.08,
17,Los Tangos,HN,80.60,15.15,-88.68,
20,Hobart,AU,82.00,-42.88,147.33,
23,Rikitea,PF,79.72,-23.12,-134.97,
26,Bambous Virieux,MU,77.00,-20.34,57.76,
32,Carutapera,BR,81.75,-1.20,-46.02,
34,San Patricio,PY,79.86,-26.98,-56.83,
35,Cockburn Town,TC,79.30,21.46,-71.14,
42,Majene,ID,81.00,-3.54,118.97,
44,Atuona,PF,80.64,-9.80,-139.03,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hilo,US,82.40,19.71,-155.08,Hilo Seaside Hotel
17,Los Tangos,HN,80.60,15.15,-88.68,
20,Hobart,AU,82.00,-42.88,147.33,St Ives Apartments
23,Rikitea,PF,79.72,-23.12,-134.97,Pension Maro'i
26,Bambous Virieux,MU,77.00,-20.34,57.76,Casa Tia Villa
32,Carutapera,BR,81.75,-1.20,-46.02,Lidera
34,San Patricio,PY,79.86,-26.98,-56.83,"Granja ""La Blanca"""
35,Cockburn Town,TC,79.30,21.46,-71.14,Osprey Beach Hotel
42,Majene,ID,81.00,-3.54,118.97,Kost Putri RABBANI
44,Atuona,PF,80.64,-9.80,-139.03,Villa Enata


In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))